In [23]:
import os
import pandas as pd
from roboflow import Roboflow
import time
from datetime import datetime
from tkinter import *
from tkinter import messagebox
from tkinter import simpledialog
from tkinter.ttk import *
from tkinter import _setit
from tkinter import filedialog
import tkinter.font as tkfont

def main():
    global model, typeHyphenCount, typeNoHyphenCount, typeIMGCount, typeUnknownCount, total, threshold

    # Set confidence threshold for contrail detection
    threshold = 40

    # Load Roboflow Models
    rf = Roboflow(api_key="QEqOPSjXRhO0tGaPxcPw")
    project = rf.workspace().project("contrails-50-50-object-det.")
    model = project.version(2).model

    # Define directory with source sky images for detection
    directory = '/Users/sarfraazshahulhameed/Downloads/Checking2'

    typeHyphenCount = 0
    typeNoHyphenCount = 0
    typeIMGCount = 0
    typeUnknownCount = 0
    total = 0

    # Open New GUI window with dimensions w, h and open in center of the screen
    global window
    window = Tk()
    w = 470
    h = 110
    ws = window.winfo_screenwidth()
    hs = window.winfo_screenheight()
    x = (ws/2) - (w/2)
    y = (hs/2) - (h/2)
    window.geometry('%dx%d+%d+%d' % (w, h, x, y))
    window.title("Contrail Detection Tool")

    def clicked_predict_new():
        user_input = simpledialog.askstring("Contrail Detection Confidence Threshold", "Enter a whole number 1-99 for the model confidence threshold. A higher number will detect fewer contrails. A lower number will detect more but may result in more false positive detections:")
        try:
            threshold = int(user_input)
            if threshold < 1 or threshold > 99:
                messagebox.showinfo("Error", "Threshold must be between 1 and 99. Please try again.")
                return
        except:
            messagebox.showinfo("Error", 'Threshold must be a whole number between 1 and 99. Please try again.')
            return
        
        print(" ")
        print("Now predicting all NEW image files in directory...")
        print(" ")
        for filename in os.listdir(directory):
            f = os.path.join(directory, filename)
            print(filename)
            print(f)

            if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                try:
                    predict_image(f, filename)
                except:
                    print("Doesn't have contrails in " + filename + " and it will be skipped.")
            else:
                print("A non-jpeg file was found in the source folder. Currently, only .jpg or .jpeg images are accepted for prediction.")
        print("Predictions complete.")

    def clicked_predict_all():
        user_input = simpledialog.askstring("Contrail Detection Confidence Threshold", "Enter a whole number 1-99 for the model confidence threshold. A higher number will detect fewer contrails. A lower number will detect more but may result in more false positive detections:")
        try:
            threshold = int(user_input)
            if threshold < 1 or threshold > 99:
                messagebox.showinfo("Error", "Threshold must be between 1 and 99. Please try again.")
                return
        except:
            messagebox.showinfo("Error", 'Threshold must be a whole number between 1 and 99. Please try again.')
            return
        print(" ")
        print("Now predicting all image files in directory...")
        print(" ")
        for filename in os.listdir(directory):
            f = os.path.join(directory, filename)
            print(filename)
            print(f)

            if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                try:
                    predict_image(f, filename)
                except:
                    print("Doesn't has contrails in " + filename + " and it will be skipped.")
            else:
                print("A non-jpeg file was found in the source folder. Currently, only .jpg or .jpeg images are accepted for prediction.")
        print("Predictions complete.")

    def clicked_delete_all():
        print("Delete all functionality not applicable for Excel.")
        # You can add Excel-related functionality here if needed.

    def clicked_export():
        df = pd.read_excel("/Users/sarfraazshahulhameed/Documents/Assignments/DAEN690/Sky Image data copy.xlsx")
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        df.to_excel("Export_" + timestamp + ".xlsx")
        print("Excel export successful and saved as Contrail Prediction Logs/Export_" + timestamp + ".xlsx")

    # Add form elements and define commands
    label = Label(window, text="   Predict and Log New Image Files Only:   ")
    label.grid(column=1, row=1, sticky=W)

    btn = Button(window, text="Predict New Files", command=clicked_predict_new)
    btn.grid(column=2, row=1, sticky=E)

    label = Label(window, text="   Predict and Log ALL Image Files:   ")
    label.grid(column=1, row=2, sticky=W)

    btn = Button(window, text="Predict All Files", command=clicked_predict_all)
    btn.grid(column=2, row=2, sticky=E)

    label = Label(window, text="   Delete All Image Predictions from Excel:   ")
    label.grid(column=1, row=3, sticky=W)

    btn = Button(window, text="Delete Predictions", command=clicked_delete_all)
    btn.grid(column=2, row=3, sticky=E)

    label = Label(window, text="   Export Current Predictions to Excel File:   ")
    label.grid(column=1, row=4, sticky=W)

    btn = Button(window, text="Export", command=clicked_export)
    btn.grid(column=2, row=4, sticky=E)

    window.mainloop()

def count_object_occurrences(predictions, target_class):
    object_count = 0
    for prediction in predictions:
        if prediction['class'] in target_class:
            object_count += 1
    return object_count

def predict_image(f, filename):
    global typeHyphenCount, typeNoHyphenCount, typeIMGCount, typeUnknownCount, total
    cirrus_ind = 0
    longlived_ind = 0
    contrail_ind = 0

    total = total + 1
    if filename[4] == "-":
        type = "hyphenated"
        typeHyphenCount = typeHyphenCount + 1
    elif filename[4] in ("0", "1"):
        type = "not_hyphenated"
        typeNoHyphenCount = typeNoHyphenCount + 1
    elif filename[0:3] == "IMG":
        type = "IMG"
        typeIMGCount = typeIMGCount + 1
    else:
        type = "unknown"
        typeUnknownCount = typeUnknownCount + 1

    if type == "hyphenated":
        year = int(filename[0:4])
        month = int(filename[5:7])
        day = int(filename[8:10])
        hour = int(filename[11:13])
        key = filename[0:4] + filename[5:7] + filename[8:10] + "_" + filename[11:13]
    elif type == "not_hyphenated":
        year = int(filename[0:4])
        month = int(filename[4:6])
        day = int(filename[6:8])
        hour = int(filename[9:11])
        key = filename[0:11]
    elif type == "IMG":
        year = int(filename[4:8])
        month = int(filename[8:10])
        day = int(filename[10:12])
        hour = int(filename[13:15])
        key = filename[4:15]
    else:
        print("Could not get date and time from file " + filename + ". Unrecognized format.")
        year = 'NULL'
        month = 'NULL'
        day = 'NULL'
        hour = 'NULL'
        key = 'UNKNOWN'

    start = datetime.now()

    # Detect image with model
    print("Model Predictions")
    predictions = model.predict(f, confidence=threshold, overlap=30)
    print(predictions)
    cirrus_count = count_object_occurrences(predictions, 'Cirrus')
    longlived_count = count_object_occurrences(predictions, 'LongLived')

    print("Cirrus Count: " + str(cirrus_count))
    print("LongLived Count: " + str(longlived_count))

    if cirrus_count > 0:
        cirrus_ind = 1

    if longlived_count > 0:
        longlived_ind = 1

    if cirrus_ind + longlived_ind > 0:
        contrail_ind = 1

    # Save out image with predicted contrails only on images which have contrails found
    if contrail_ind == 1:
        total_contrail_count = cirrus_count + longlived_count
        folder_path = f"Predicted Sky Images/{total_contrail_count} Contrails"
        save_path = f"{folder_path}/predict_{filename}"
    
    # Check if the directory exists, if not, create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Use your model to predict and save the image
    model.predict(f, confidence=40, overlap=30).save(save_path)

    end = datetime.now()
    prediction_time = (end - start).total_seconds()


    # Log results in Excel file
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    log_df = pd.DataFrame({
    'IMAGE_FILENAME': [filename],
        'YEAR': [year],
        'MONTH': [month],
        'DAY': [day],
        'HOUR': [hour],
        'CIRRUS_CONTRAIL_CT': [cirrus_count],
        'CIRRUS_CONTRAIL_IND': [cirrus_ind],
        'LONG_LIVED_CONTRAIL_CT': [longlived_count],
        'LONG_LIVED_CONTRAIL_IND': [longlived_ind],
        'CONTRAIL_IND': [contrail_ind],
        'PREDICTION_TIME': [prediction_time]
    })
        # Append to the Excel file
    log_df.to_excel("/Users/sarfraazshahulhameed/Documents/Assignments/DAEN690/Sky Image data copy.xlsx", index=False, header=False, mode='a')


if __name__ == "__main__":
    main()


loading Roboflow workspace...
loading Roboflow project...
Excel export successful and saved as Contrail Prediction Logs/Export_20231129-135112.xlsx
 
Now predicting all NEW image files in directory...
 
IMG_20221219_090048.jpg
/Users/sarfraazshahulhameed/Downloads/Checking2/IMG_20221219_090048.jpg
Model Predictions

Cirrus Count: 0
LongLived Count: 0
Doesn't have contrails in IMG_20221219_090048.jpg and it will be skipped.
IMG_20221216_130048.jpg
/Users/sarfraazshahulhameed/Downloads/Checking2/IMG_20221216_130048.jpg
Model Predictions

Cirrus Count: 0
LongLived Count: 0
Doesn't have contrails in IMG_20221216_130048.jpg and it will be skipped.
IMG_20221219_100048.jpg
/Users/sarfraazshahulhameed/Downloads/Checking2/IMG_20221219_100048.jpg
Model Predictions
{
  "x": 2721,
  "y": 503,
  "width": 352,
  "height": 874,
  "confidence": 0.8423409461975098,
  "class": "LongLived",
  "class_id": 1,
  "image_path": "/Users/sarfraazshahulhameed/Downloads/Checking2/IMG_20221219_100048.jpg",
  "pred